# Урок 8. Generative Adversarial Networks
 Домашнее задание
Обучить генератор создавать точки, которые будут лежать на графике функции 𝑦=sin(𝑥)/𝑥−𝑥/10
При выполнении данного задания структура GAN остается той же, но Вам нужно:
Сгенерировать настоящие данные
Изменить архитектуру дискриминатора и генератора
Построить графики

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython import display

In [2]:
device = 'cpu'
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
df = pd.DataFrame(np.linspace(-5, 5, 100), columns=['x'])
df['y'] = np.sin(df['x']) / df['x'] - (df['x'] / 10)
df.head()

,x,y
0,-20.00000,2.045647
1,-19.95996,2.040880
2,-19.91992,2.036037
3,-19.87988,2.031120
4,-19.83984,2.026131


In [ ]:
plt.plot(df['x'], df['y'])
plt.show()

In [ ]:
# Class Generator

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.activations = nn.ModuleDict([
            ['lrelu', nn.LeakyReLU(0.2, inplace=True)],
            ['relu', nn.ReLU()]
        ])

        def block(in_feat, out_feat, normalize=True, activation='relu'):
            layers = [nn.Linear(in_feat, out_feat)]  

            if normalize:
                layers.append(nn.BatchNorm1d(out_feat))
            layers.append(self.activations[activation])  
            
            return layers

        self.model = nn.Sequential(
      
            *block(latent_dim, 128, normalize=False), 
            *block(128, 256, activation='lrelu'),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 2),
        )
        

    def forward(self, z):
        num = self.model(z)

        return num

# Class CustomLinearLayer

class CustomLinearLayer(nn.Module):
    def __init__(self, size_in, size_out):

        super().__init__()

        self.size_in, self.size_out = size_in, size_out
        
        weights = torch.Tensor(size_out, size_in)

        self.weights = nn.Parameter(weights) 

        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)
        
        nn.init.uniform_(self.weights, -0.5, 0.5) 
        nn.init.uniform_(self.bias, -0.5, 0.5)  

    def forward(self, x):
        w_times_x = torch.mm(x, self.weights.t())
        return torch.add(w_times_x, self.bias)

# Class Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            CustomLinearLayer(2, 512),
            nn.LeakyReLU(0.2, inplace=True),
            CustomLinearLayer(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            CustomLinearLayer(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, pred):
        validity = self.model(pred)

        return validity

In [ ]:
n_epochs = 100
batch_size = 32
lr = 0.0005

b1 = 0.5
b2 = 0.999

latent_dim = 1000

sample_interval = 100

# df в массив

real_data = torch.utils.data.DataLoader(
    df.to_numpy(), # to_numpy()
    batch_size=batch_size,
    shuffle=True)

real_data

In [ ]:
# Generator, Discriminator

generator = Generator().to(device)
discriminator = Discriminator().to(device)

optimizer_G = torch.optim.Adam(
    generator.parameters(), 
    lr=lr, 
    betas=(b1, b2)
)
optimizer_D = torch.optim.Adam(
    discriminator.parameters(), 
    lr=lr, 
    betas=(b1, b2)
)

adversarial_loss = torch.nn.BCELoss()

red_patch = mpatches.Patch(color='red', label='D loss')
green_patch = mpatches.Patch(color='green', label='G loss')

In [ ]:

d_loss_history = []
g_loss_history = []

for epoch in range(n_epochs):
    for i, data in enumerate(real_data):
        
        cur_batch_size = len(data)


        valid = torch.FloatTensor(cur_batch_size, 1).fill_(1.0).to(device)
  
        fake = torch.FloatTensor(cur_batch_size, 1).fill_(0.0).to(device)

        z = Variable(torch.FloatTensor(np.random.uniform(-50, 50, (cur_batch_size, latent_dim)))).to(device)
        
        real_ex = Variable(data.type(torch.FloatTensor)).to(device)

        gen_data = generator(z)


        real_pred = discriminator(real_ex)

        d_real_loss = adversarial_loss(real_pred, valid)

        fake_pred = discriminator(gen_data.detach())

        d_fake_loss = adversarial_loss(fake_pred, fake)
        
        d_loss = (d_real_loss + d_fake_loss) / 2

        optimizer_D.zero_grad()
        
        d_loss.backward()
        
        optimizer_D.step()
        
        fake_pred = discriminator(gen_data)
        
        g_loss = adversarial_loss(fake_pred, valid)
        
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()
        
        batches_done = epoch * len(real_data) + i
    
        if batches_done % sample_interval == 0:
            plt.clf()
            
            display.clear_output(wait=False)
            print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"% (epoch, n_epochs, i, len(real_data), d_loss.item(), g_loss.item()) ) 

            display.display(plt.gcf())

            d_loss_history.append(d_loss.detach().numpy())
            g_loss_history.append(g_loss.detach().numpy())
            
            plt.plot(np.log(np.array(d_loss_history)), label='D loss', color = 'red')
            plt.plot(np.log(np.array(g_loss_history)), label='G loss', color = 'green')

            plt.legend(handles=[red_patch, green_patch])
            plt.show()

In [ ]:
generator.eval()

test_z = Variable(torch.from_numpy(np.random.uniform(-40, 35, (100, latent_dim)).astype(np.float64)).float()).to(device)

with torch.no_grad():
    test_pred = generator(test_z)

In [ ]:
nt = 0
error = 0.0
count = 0
for i in range(len(test_pred)):
    x = test_pred[i][0]
    y_real = (np.sin(x) / x) - (x / 10) 
    y_pred = test_pred[i][1]  
    count +=1
    error += abs(y_real - y_pred)

print("Mean error: %.10f" % (error/count))

In [ ]:
plt.figure()

x = np.linspace(-25, 25, 1000)
plt.plot(x, ((np.sin(x) / x) - (x / 10) ), label='y', color = 'red')

p = test_pred[:,0].argsort()
plt.plot(test_pred[:,0][p], test_pred[:, 1][p], label='y pred', color = 'blue')

plt.legend()
plt.show()